#### Report Usage Datasets 

##### Data ingestion strategy:
<mark style="background: #88D5FF;">**N/A**</mark>

##### Related pipeline:

**Ext_Load_PBI_Report_Usage_E2E**

##### Source:

**Table** from FUAM_Ext_Lakehouse table **gold_table_name** variable

##### Target:

**1 Data pipeline** in related_pipeline 
- **odata_response_json** variable value returned by notebook

In [ ]:
import json
from pyspark.sql import SparkSession
from notebookutils import mssparkutils # type: ignore

print("Successfully imported all packages for this notebook.")

In [ ]:
#
# Create the Spark session
#
app_name = "GetReportUsageDatasets"

# Get the current Spark session
spark = SparkSession.builder \
    .appName(app_name) \
    .getOrCreate()

print(f"Spark session {app_name} has been created successfully.")

In [ ]:
## Parameters
display_data = True

## Variables
gold_table_name = "workspace_datasets"

print("Successfully configured all paramaters for this run.")

In [ ]:
# Read from the Lakehouse table
df = spark.read.table(gold_table_name)

# Filter by DatasetName
# filtered_df = df.filter(df["DatasetName"] == "Report Usage Metrics Model") # Old usage report format
filtered_df = df.filter(df["DatasetName"] == "Usage Metrics Report") # New usage report format

# Select only needed columns
selected_df = filtered_df.select("WorkspaceId", "DatasetId")

print(f"Gold layer table {gold_table_name} has been read and filtered successfully.")

In [ ]:
if display_data:
    display(selected_df)

In [ ]:
# Collect the values to a list of dicts
results = selected_df.rdd.map(lambda row: {"WorkspaceId": row["WorkspaceId"], "DatasetId": row["DatasetId"]}).collect()

# Wrap the results in the desired format
odata_response = {
    "@odata.context": "https://wabi-us-east2-d-primary-redirect.analysis.windows.net/v1.0/myorg/admin/$metadata#groups",
    "@odata.count": len(results),
    "value": results
}

# Convert the response to a JSON string
odata_response_json = json.dumps(odata_response)
if display_data:
   print(odata_response_json)

In [ ]:
type(odata_response_json)

In [ ]:
print("Successfully created the result JSON which will be returned to the pipeline.")

# Send result to pipeline variable
mssparkutils.notebook.exit(odata_response_json)